In [9]:
from keras import layers

from models import create_model
from models.layers import RandomSwitch
from utils import plot_metrics
from utils.data import load_data, over_sampling, random_resample

data_path = 'data/chest_xray'

In [10]:
resize = (224, 244)

In [11]:
(x, y) = load_data(data_path, resize=resize, resample=random_resample)

In [12]:
NEG_IDX = 0
POS_IDX = 1
TOTAL   = len(y)
N       = len(list(filter(lambda j: j[NEG_IDX] > 0, y)))
P       = len(list(filter(lambda j: j[POS_IDX] > 0, y)))
print(f'#Positives = {P} ({P/TOTAL*100:.2f}%)')
print(f'#Negatives = {N} ({N/TOTAL*100:.2f}%)')

#Positives = 4273 (72.97%)
#Negatives = 1583 (27.03%)


In [13]:
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = train_test_split(x, y, test_size=0.2, random_state=42)
total_train = len(Y_train)
N_train     = len(list(filter(lambda j: j[NEG_IDX] > 0, Y_train)))
P_train     = len(list(filter(lambda j: j[POS_IDX] > 0, Y_train)))
print(f'#Positives (train) = {P_train} ({P_train/total_train*100:.2f}%)')
print(f'#Negatives (train) = {N_train} ({N_train/total_train*100:.2f}%)')
total_val   = len(Y_val)
N_val       = len(list(filter(lambda j: j[NEG_IDX] > 0, Y_val)))
P_val       = len(list(filter(lambda j: j[POS_IDX] > 0, Y_val)))
print(f'#Positives (val)   = {P_val} ({P_val/total_val*100:.2f}%)')
print(f'#Negatives (val)   = {N_val} ({N_val/total_val*100:.2f}%)')

#Positives (train) = 3408 (72.76%)
#Negatives (train) = 1276 (27.24%)
#Positives (val)   = 865 (73.81%)
#Negatives (val)   = 307 (26.19%)


In [14]:
xs_extras, ys_extras = over_sampling(X_train, Y_train)
total   = len(ys_extras)
N_extras       = len(list(filter(lambda j: j[NEG_IDX] > 0, ys_extras)))
P_extras       = len(list(filter(lambda j: j[POS_IDX] > 0, ys_extras)))
print(f'#Positives (val)   = {P_extras} ({P_extras/total*100:.2f}%)')
print(f'#Negatives (val)   = {N_extras} ({N_extras/total*100:.2f}%)')

#Positives (val)   = 3408 (50.00%)
#Negatives (val)   = 3408 (50.00%)


In [15]:
tr = layers.RandomTranslation(0.2, 0.2, data_format='channels_last')
rr = layers.RandomRotation(0.2, data_format='channels_last')
rb = layers.RandomBrightness(0.2, data_format='channels_last')
rc = layers.RandomContrast(0.2, data_format='channels_last')
rz = layers.RandomZoom(0.2, 0.2, data_format='channels_last')
p1 = layers.Pipeline([rr, rz])
p2 = layers.Pipeline([tr, rr])
p3 = layers.Pipeline([rr, rc])
data_augmentation = RandomSwitch([tr, rr, rb, rc, rz, p1, p2, p3], 0.7)

In [16]:
v1 = create_model('resnet_v1', resize=resize, data_augmentation=data_augmentation)

In [17]:
from keras import losses, metrics, optimizers

METRICS = [
    metrics.BinaryCrossentropy(name='cross entropy'),  # same as model's loss
    # metrics.MeanSquaredError(name='Brier score'),
    # metrics.TruePositives(name='tp'),
    # metrics.FalsePositives(name='fp'),
    # metrics.TrueNegatives(name='tn'),
    # metrics.FalseNegatives(name='fn'),
    # metrics.BinaryAccuracy(name='accuracy'),
    metrics.Precision(name='precision'),
    metrics.Recall(name='recall'),
    metrics.AUC(name='auc'),
    metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

v1.compile(optimizer=optimizers.Adam(), loss=losses.BinaryCrossentropy(), metrics=METRICS)
v1.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ random_switch_1 (RandomSwitch)  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_1 (Rescaling)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ stem_v1_1 (StemV1)              │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_a_vx_5 (InceptionAVx) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_a_vx_6 (InceptionAVx) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_a_vx_7 (InceptionAVx) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_a_vx_8 (InceptionAVx) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_a_vx_9 (InceptionAVx) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reduction_a_1 (ReductionA)      │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_w_vx_15               │ ?                      │   0 (unbuilt) │
│ (InceptionWVx)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_w_vx_16               │ ?                      │   0 (unbuilt) │
│ (InceptionWVx)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_w_vx_17               │ ?                      │   0 (unbuilt) │
│ (InceptionWVx)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_w_vx_18               │ ?                      │   0 (unbuilt) │
│ (InceptionWVx)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_w_vx_19               │ ?                      │   0 (unbuilt) │
│ (InceptionWVx)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_w_vx_20               │ ?                      │   0 (unbuilt) │
│ (InceptionWVx)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_w_vx_21               │ ?                      │   0 (unbuilt) │
│ (InceptionWVx)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_w_vx_22               │ ?                      │   0 (unbuilt) │
│ (InceptionWVx)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_w_vx_23               │ ?                      │   0 (unbuilt) │
│ (InceptionWVx)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_w_vx_24               │ ?                      │   0 (unbuilt) │
│ (InceptionWVx)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reduction_b_vx_1 (ReductionBVx) │ ?                      │   0 (unbuilt

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [18]:
from keras import callbacks

EPOCHS = 100
BATCH_SIZE = 128
early_stopping = callbacks.EarlyStopping(
    verbose=1,
    patience=10,
    restore_best_weights=True)

In [19]:
history = v1.fit(
    xs_extras,
    ys_extras,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[early_stopping],
    validation_data=(X_val, Y_val),
    validation_freq=1)

Epoch 1/100


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f8323c69400>>
Traceback (most recent call last):
  File "/home/francesco/Documents/Università/ML/t5/.venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
plot_metrics(history, metrics=['loss', 'precision', 'recall', 'auc'])